In [ ]:
!pip install -q -U transformers datasets bitsandbytes  trl peft  huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# model_name = "Cagatayd/llama3.2-1B-Instruct-Egitim"
# model_name = "unsloth/Llama-3.2-1B-Instruct"
model_name = "meta-llama/Llama-3.2-1B-Instruct"

config_8bit = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(


                                                  model_name,
                                                  quantization_config=config_8bit,
                                                  device_map= "auto",
                                                  trust_remote_code =True

                                                  )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
model_8bit

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMS

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,padding_side="left",trust_remote_code= True)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset("MattCoddity/dockerNLcommands")
dataset

README.md:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

06102023.json:   0%|          | 0.00/543k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2415 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 2415
    })
})

In [ ]:
dataset['train'][0]

{'input': 'Give me a list of containers that have the Ubuntu image as their ancestor.',
 'output': "docker ps --filter 'ancestor=ubuntu'",
 'instruction': 'translate this sentence in docker command'}

In [ ]:
from datasets import DatasetDict

train_val_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

dataset = DatasetDict(

                      {'train':train_val_split['train'],
                       'validation':train_val_split['test']}

)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 483
    })
})

In [ ]:
def to_chat_template(example):


    messages = [

                {"role": 'system','content':example['instruction']},
                {"role": 'user','content':example['input']},
                {"role": 'assistant','content':example['output']}

                ]


    return {'text':messages}

dataset = dataset.map(to_chat_template)
dataset



Map:   0%|          | 0/1932 [00:00<?, ? examples/s]

Map:   0%|          | 0/483 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 483
    })
})

In [ ]:
dataset['train']['text'][0]

[{'content': 'translate this sentence in docker command', 'role': 'system'},
 {'content': 'Find the repository, tag, and ID of the images that were created before the latest nginx image.',
  'role': 'user'},
 {'content': 'docker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"',
  'role': 'assistant'}]

In [ ]:
tokenizer_mistral = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer_mistral.apply_chat_template(dataset['train']['text'][0],tokenize=False)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

'<s> [INST] translate this sentence in docker command\n\nFind the repository, tag, and ID of the images that were created before the latest nginx image. [/INST] docker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"</s>'

In [ ]:
tokenizer.apply_chat_template(dataset['train']['text'][0],tokenize=False)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 21 Feb 2025\n\ntranslate this sentence in docker command<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nFind the repository, tag, and ID of the images that were created before the latest nginx image.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\ndocker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"<|eot_id|>'

In [ ]:
def apply_chat_temp(example):

    new_text = tokenizer.apply_chat_template(example['text'],tokenize=False)

    return {'text':new_text}


dataset = dataset.map(apply_chat_temp)
dataset

Map:   0%|          | 0/1932 [00:00<?, ? examples/s]

Map:   0%|          | 0/483 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 483
    })
})

In [ ]:
dataset['train']['text'][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 21 Feb 2025\n\ntranslate this sentence in docker command<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nFind the repository, tag, and ID of the images that were created before the latest nginx image.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\ndocker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"<|eot_id|>'

In [ ]:
tokenizer(dataset['train']['text'][0])

{'input_ids': [128000, 128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1691, 13806, 220, 2366, 20, 271, 14372, 420, 11914, 304, 27686, 3290, 128009, 128006, 882, 128007, 271, 10086, 279, 12827, 11, 4877, 11, 323, 3110, 315, 279, 5448, 430, 1051, 3549, 1603, 279, 5652, 71582, 2217, 13, 128009, 128006, 78191, 128007, 271, 29748, 5448, 482, 69, 330, 15145, 28, 74661, 25, 19911, 1, 1198, 2293, 48319, 13, 4727, 39254, 3052, 13, 5786, 39254, 3052, 13, 926, 24275, 128009], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def tokenize_fn(example):

    return tokenizer(example['text'])


tokenized_dataset = dataset.map(tokenize_fn,batched=True,remove_columns=['input', 'output', 'instruction', 'text'])
tokenized_dataset

Map:   0%|          | 0/1932 [00:00<?, ? examples/s]

Map:   0%|          | 0/483 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 483
    })
})

In [ ]:
len(tokenized_dataset['train']['input_ids'][1])

79

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False,return_tensors="pt")


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

'<|eot_id|>'

In [ ]:
from torch.utils.data import DataLoader


data_loader = DataLoader(
    tokenized_dataset['train'],
    batch_size=2,
    collate_fn=data_collator
)
for step, batch in enumerate(data_loader):
    print(f"Batch {step}")
    print("input_ids.shape:", batch["input_ids"].shape)

    print("attention_mask.shape:", batch["attention_mask"].shape)


    if step >= 3:
        break

Batch 0
input_ids.shape: torch.Size([2, 88])
attention_mask.shape: torch.Size([2, 88])
Batch 1
input_ids.shape: torch.Size([2, 79])
attention_mask.shape: torch.Size([2, 79])
Batch 2
input_ids.shape: torch.Size([2, 80])
attention_mask.shape: torch.Size([2, 80])
Batch 3
input_ids.shape: torch.Size([2, 78])
attention_mask.shape: torch.Size([2, 78])


In [ ]:
# 2 2

# [1,2,3,4,5,6]
# [pad,pad,pad,1,2,3] -100



# [1,2,3,4,5]
# [pad,pad,1,2,3]


In [ ]:
model_8bit

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMS

In [ ]:
from peft import  LoraConfig, get_peft_model
import copy

model_8bit_clone = copy.deepcopy(model_8bit)


lora_config = LoraConfig(

                         r= 64,
                         lora_alpha = 32,
                         target_modules = ["q_proj","k_proj","v_proj","o_proj",
                      "gate_proj","up_proj","down_proj"],
                         lora_dropout = 0.05,
                         bias = "none",
                         task_type = "CAUSAL_LM"

)

model_8bit_lora = get_peft_model(model_8bit_clone,lora_config)
model_8bit_lora.print_trainable_parameters()

trainable params: 45,088,768 || all params: 1,280,903,168 || trainable%: 3.5201


In [ ]:
model_8bit

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMS

In [ ]:
model_8bit_lora

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer


training_args = TrainingArguments(

                                  output_dir = "./resutls",
                                  per_device_train_batch_size = 2,
                                  per_device_eval_batch_size = 2,
                                  eval_steps = 10,
                                  evaluation_strategy = "steps",
                                  save_steps = 20,
                                  save_strategy = "steps",
                                  # num_train_epochs = 1,
                                  max_steps = 60,
                                  learning_rate = 3e-5,
                                  weight_decay = 0.01,
                                  warmup_steps = 5,
                                  fp16 = True,
                                  gradient_accumulation_steps = 4,
                                  optim = "adamw_torch",
                                  logging_steps = 10,
                                  report_to = "none"


                                  )

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 483
    })
})

In [ ]:
trainer = SFTTrainer(

                     model =model_8bit_lora,
                     train_dataset =tokenized_dataset['train'],
                     eval_dataset = tokenized_dataset['validation'],
                     args = training_args,
                     data_collator= data_collator,
                     tokenizer = tokenizer
)

trainer.train()

<ipython-input-30-046adc83ed52>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/1932 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1932 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1932 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/483 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/483 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/483 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
10,4.383600,3.290534
20,2.704600,2.147171
30,1.908200,1.652376
40,1.461900,1.330681
50,1.286300,1.208860
60,1.197200,1.184752


TrainOutput(global_step=60, training_loss=2.1569530646006267, metrics={'train_runtime': 252.0598, 'train_samples_per_second': 1.904, 'train_steps_per_second': 0.238, 'total_flos': 226414707916800.0, 'train_loss': 2.1569530646006267})

In [ ]:
model_8bit_lora

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [ ]:
import torch

torch.set_printoptions(precision=8, sci_mode=False)

layer0 = model_8bit_lora.model.model.layers[0]

model_8bit_weights = layer0.self_attn.q_proj.weight
print("model_8bit_weights:", model_8bit_weights)

model_8bit_weights: Parameter containing:
Parameter(Int8Params([[-44,  16,  61,  ..., -22, -29,  50],
            [ 14,  70,  65,  ..., -39, -18,  13],
            [ 16,  14,  30,  ..., -34, -34, -24],
            ...,
            [ 17,  20,  40,  ..., -40, -15, -16],
            [ 32, -35,  50,  ..., -17, -41, -21],
            [-14, -30,  -7,  ...,  30,   5,  -2]], device='cuda:0',
           dtype=torch.int8))


In [ ]:
import torch

torch.set_printoptions(precision=8, sci_mode=False)

layer0 = model_8bit_lora.model.model.layers[0]

# LoRA A matrisi
weight_A = layer0.self_attn.q_proj.lora_A["default"].weight
print("LoRA A weight:", weight_A)

LoRA A weight: Parameter containing:
tensor([[ 0.01543376, -0.01957894,  0.01014431,  ...,  0.00173371,
          0.01613926, -0.01556054],
        [-0.01690101,  0.01074822,  0.01061925,  ...,  0.00292684,
         -0.00149638, -0.01090698],
        [-0.01426101,  0.01581102,  0.01960918,  ...,  0.00724101,
          0.01115141, -0.00878092],
        ...,
        [-0.01080369,  0.00528889, -0.01511446,  ..., -0.00140287,
          0.01316711, -0.01059285],
        [ 0.01593476,  0.01215826, -0.01826999,  ...,  0.01343037,
         -0.01595012, -0.00277690],
        [ 0.01049892, -0.00961566, -0.01900158,  ...,  0.00369666,
          0.00430331, -0.01020450]], device='cuda:0', requires_grad=True)


In [ ]:
weight_A.dtype

torch.float32

In [ ]:

base_model = AutoModelForCausalLM.from_pretrained(


                                                  model_name,

                                                  device_map= "auto",
                                                  trust_remote_code =True

                                                  )


In [ ]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [ ]:
lora_path = "./lora_adapters"
model_8bit_lora.save_pretrained(lora_path)

In [ ]:
from peft import PeftModel

base_model =PeftModel.from_pretrained(base_model,lora_path)
base_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [ ]:
base_model = base_model.merge_and_unload()
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [ ]:
base_model.push_to_hub("Cagatayd/Llama3.2-doker-egitim")
tokenizer.push_to_hub("Cagatayd/Llama3.2-doker-egitim")

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Cagatayd/Llama3.2-doker-egitim/commit/883a2f201eb8efc3262705c890c88a6ba70ae3a0', commit_message='Upload tokenizer', commit_description='', oid='883a2f201eb8efc3262705c890c88a6ba70ae3a0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Cagatayd/Llama3.2-doker-egitim', endpoint='https://huggingface.co', repo_type='model', repo_id='Cagatayd/Llama3.2-doker-egitim'), pr_revision=None, pr_num=None)

In [ ]:
# import torch
# from transformers import pipeline

# model_id = "Cagatayd/Llama3.2-doker-egitim"
# pipe = pipeline(
#     "text-generation",
#     model=model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
# messages = [
#     {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
#     {"role": "user", "content": "Who are you?"},
# ]
# outputs = pipe(
#     messages,
#     max_new_tokens=256,
# )
# print(outputs[0]["generated_text"][-1])


In [ ]:
dataset['train'][1]

{'input': 'Please show me the Docker containers that have exited and are related to the mongo image.',
 'output': "docker ps -a --filter 'status=exited' --filter 'ancestor=mongo'",
 'instruction': 'translate this sentence in docker command',
 'text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 21 Feb 2025\n\ntranslate this sentence in docker command<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPlease show me the Docker containers that have exited and are related to the mongo image.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\ndocker ps -a --filter 'status=exited' --filter 'ancestor=mongo'<|eot_id|>"}

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=base_model,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    tokenizer = tokenizer
)
messages = [
    {"role": "system", "content": "translate this sentence in docker command"},
    {"role": "user", "content": "Please show me the Docker containers that have exited and are related to the mongo image."},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1]['content'])


Device set to use cuda:0


docker ps --filter "status=exited" --filter "image= mongo"


In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=base_model,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    tokenizer = tokenizer
)
messages = [
    {"role": "system", "content": "translate this sentence in docker command"},
    {"role": "user", "content": "Please show me the Docker containers that have exited and are related to the mongo image."},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
    temperature = 0.5,
    top_p  = 0.9,
    top_k = 10,
    do_sample = True,
    repetition_penalty = 1.2


)
print(outputs[0]["generated_text"][-1]['content'])

Device set to use cuda:0


docker ps --filter "status=exited" --filter "image=mongo"
